In [1]:
import sys
import os
from pathlib import Path
project_root = os.path.join('/home/shashank/research/hyper/HyperPPO')
os.chdir(project_root)
%pwd 
import torch
v = torch.ones(1, device='cuda')  # init torch cuda before jax

from IPython.display import HTML
from IPython.display import display
from brax.io import html as brax_html


from sample_factory.enjoy import *
from sf_examples.brax.train_brax import register_brax_custom_components, parse_sf_args, add_extra_params_func, override_default_params_func, parse_full_cfg, BraxEnv
from sample_factory.algo.utils.make_env import BatchedDictObservationsWrapper


/home/shashank/miniconda3/envs/hyper/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from brax.io.json import brax, List, Optional, Text, json_format, JaxEncoder, json



def render(sys: brax.System,
           qps: List[brax.QP],
           height: int = 480,
           info: Optional[brax.Info] = None) -> str:
  """Returns an HTML page that visualizes the system and qps trajectory."""
  # if any((len(qp.pos.shape), len(qp.rot.shape)) != (2, 2) for qp in qps):
  #   raise RuntimeError('unexpected shape in qp.')
  system = dumps(sys, qps, info)
  html = brax_html._HTML.replace('<!-- system json goes here -->', system)
  html = html.replace('<!-- viewer height goes here -->', f'{height}px')
  return html


def dumps(sys: brax.System,
          qps: List[brax.QP],
          info: Optional[List[brax.Info]] = None) -> Text:
  """Creates a json string of the system config."""
  d = {
      'config': json_format.MessageToDict(sys.config, True),
      'pos': [qp.pos[0] for qp in qps],
      'rot': [qp.rot[0] for qp in qps],
      'debug': info is not None,
  }
  if info:
    # Add debug info for the contact points.
    max_len = max([sum(np.array(i.contact_penetration) > 0) for i in info])

    def _pad_arr(arr):
      arr = np.array(arr)
      padding = -np.ones(max_len - arr.shape[0])
      if len(arr.shape) > 1:
        padding = -np.ones((max_len - arr.shape[0], arr.shape[1]))
      return np.concatenate([arr, padding])

    # Pad the contact points.
    d['contact_pos'] = [
        _pad_arr(i.contact_pos[i.contact_penetration > 0]) for i in info
    ]
    d['contact_normal'] = [
        _pad_arr(i.contact_normal[i.contact_penetration > 0]) for i in info
    ]
    d['contact_penetration'] = [
        _pad_arr(i.contact_penetration[i.contact_penetration > 0]) for i in info
    ]
  return json.dumps(d, cls=JaxEncoder)

In [3]:
env = "humanoid"
hyper = "True"
train_dir = "train_dir/hyper/hyper_"
experiment = "08_hyper_see_8561347_env_humanoid_d.cri_False_m.std_True_a.s.mod_biased"
# milestone_name = "checkpoint_000198558_152616960.pth"
milestone_name = None

In [4]:
def parse_brax_cfg(eval_str, evaluation=False):
    parser, partial_cfg = parse_sf_args(evaluation=evaluation, argv=eval_str)
    add_extra_params_func(parser)
    override_default_params_func(partial_cfg.env, parser)
    final_cfg = parse_full_cfg(parser,eval_str)
    return final_cfg

In [5]:
eval_str = "python -m sf_examples.brax.enjoy_hyper_brax"
eval_str += f" --env {env}"
eval_str += f" --train_dir {train_dir}"
eval_str += f" --experiment {experiment}"

argv = eval_str.split()[3:]

register_brax_custom_components(evaluation=False)
cfg = parse_brax_cfg(evaluation=True, eval_str=argv)

In [6]:
verbose = True

cfg = load_from_checkpoint(cfg)

from brax import envs

gym_env = envs.create_gym_env(env_name=env, batch_size=2, seed=0, backend="gpu")
env = BatchedDictObservationsWrapper(BraxEnv(gym_env, 2, None, cfg.brax_render_res, cfg.clamp_actions, cfg.clamp_rew_obs))

[2023-08-24 19:07:44,367][46135] Loading existing experiment configuration from train_dir/hyper/hyper_/08_hyper_see_8561347_env_humanoid_d.cri_False_m.std_True_a.s.mod_biased/config.json
[2023-08-24 19:07:44,368][46135] Overriding arg 'train_dir' with value 'train_dir/hyper/hyper_' passed from command line
[2023-08-24 19:07:44,368][46135] Adding new argument 'fps'=0 that is not in the saved config file!
[2023-08-24 19:07:44,369][46135] Adding new argument 'eval_env_frameskip'=None that is not in the saved config file!
[2023-08-24 19:07:44,369][46135] Adding new argument 'no_render'=False that is not in the saved config file!
[2023-08-24 19:07:44,370][46135] Adding new argument 'save_video'=False that is not in the saved config file!
[2023-08-24 19:07:44,370][46135] Adding new argument 'video_frames'=1000000000.0 that is not in the saved config file!
[2023-08-24 19:07:44,371][46135] Adding new argument 'video_name'=None that is not in the saved config file!
[2023-08-24 19:07:44,371][461

In [7]:
actor_critic = create_actor_critic(cfg, env.observation_space, env.action_space)
actor_critic.eval()

device = torch.device("cpu" if cfg.device == "cpu" else "cuda")
actor_critic.model_to_device(device)

policy_id = cfg.policy_index
name_prefix = dict(latest="checkpoint", best="best")[cfg.load_checkpoint_kind]
checkpoints = Learner.get_checkpoints(Learner.checkpoint_dir(cfg, policy_id), f"{name_prefix}_*")


checkpoint_dict = Learner.load_checkpoint(checkpoints, device)
actor_critic.load_state_dict(checkpoint_dict["model"])



[2023-08-24 19:07:47,373][46135] RunningMeanStd input shape: (240,)
[2023-08-24 19:07:47,393][46135] RunningMeanStd input shape: (1,)
[2023-08-24 19:07:49,831][46135] Loading state from checkpoint train_dir/hyper/hyper_/08_hyper_see_8561347_env_humanoid_d.cri_False_m.std_True_a.s.mod_biased/checkpoint_p0/checkpoint_000030544_1000865792.pth...


<All keys matched successfully>

In [8]:
list_of_test_archs = [
                [4],                                   
            ]
ghn = actor_critic.actor_encoder.ghn
ghn.eval()    

list_of_test_arch_indices = [[i for i,arc in enumerate(actor_critic.actor_encoder.list_of_arcs) if list(arc) == t_arc][0] for t_arc in list_of_test_archs]
list_of_test_shape_inds = torch.stack([actor_critic.actor_encoder.list_of_shape_inds[index][0:11] for k,index in enumerate(list_of_test_arch_indices)])

actor_critic.actor_encoder.set_graph(list_of_test_arch_indices, list_of_test_shape_inds)
test_arch_policy = actor_critic.actor_encoder.current_model[0]


In [9]:
episode_rewards = np.zeros((env.num_agents))
num_frames = 0

def max_frames_reached(frames):
    return cfg.max_num_frames is not None and frames > cfg.max_num_frames

reward_list = []

obs, infos = env.reset()
rnn_states = torch.zeros([env.num_agents, get_rnn_size(cfg)], dtype=torch.float32, device=device)
episode_reward = None
finished_episode = [False for _ in range(env.num_agents)]

video_frames = []
num_episodes = 0
rollout = [env.unwrapped.env._state]

with torch.no_grad():
    while not all(finished_episode):
        normalized_obs = prepare_and_normalize_obs(actor_critic, obs)

        if not cfg.no_render:
            visualize_policy_inputs(normalized_obs)
        
        actions = test_arch_policy(normalized_obs['obs'])
        

        obs, rew, terminated, truncated, infos = env.step(actions)
        dones = make_dones(terminated, truncated)
        rollout.append(env.unwrapped.env._state)

        if episode_reward is None:
            episode_reward = rew.float().clone()
        else:
            episode_reward += rew.float()

        dones = dones.cpu().numpy()
        for agent_i, done_flag in enumerate(dones):
            if done_flag:
                finished_episode[agent_i] = True
                rew = episode_reward[agent_i].item()
                episode_rewards[agent_i] = rew
                episode_reward[agent_i] = 0


    if all(finished_episode):
        print(f"test reward:", episode_rewards.mean())

i = HTML(render(env.unwrapped.env._env.sys, [s.qp for s in rollout]))
display(i)

env.close()



test reward: 11748.400390625
